## Interpolates a folder of single-frame Sacks-like calendars to a reference grid

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import tqdm
import os
import re
import copy
import datetime

In [2]:
calname = "Sacks_ZARC_fill_fill_120d"
infolder = "calendars_sacks/"+calname+"/"
outfolder = "bcagcfsr_calendars/"+calname+"/"
reffname = "D:/data/bcAgCFSR/regrid/bcAgCFSR_tavg_2001.nc4"

infsuf = ".crop.calendar.fill.nc"
crops = ["Maize","Soybeans","Cotton"]

In [3]:
# Create output folder if it doesn't exist
if not os.path.exists(outfolder): os.makedirs(outfolder, exist_ok=True)

In [4]:
# Open reference array
# refarr = xr.open_dataarray(reffname)
refds = xr.open_dataset(reffname, decode_times = False)
refarr = refds["tavg"]
print(refarr)

<xarray.DataArray 'tavg' (time: 365, lat: 360, lon: 720)>
[94608000 values with dtype=float32]
Coordinates:
  * time     (time) float32 7671.0 7672.0 7673.0 7674.0 ... 8033.0 8034.0 8035.0
  * lon      (lon) float32 -179.75 -179.25 -178.75 ... 178.75 179.25 179.75
  * lat      (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
Attributes:
    units:               Degrees Celsius
    description:         Average Temperature
    vMin_original_data:  -72.3
    vMax_original_data:  43.1
    vRange:              115.4


In [6]:
# Loop here
for crop in crops:
    # Open the dataset
    infname = infolder + crop + infsuf
    inarr = xr.open_dataset(infname)
    
    # Rename coordinates
    inarr = inarr.rename({"latitude":"lat", "longitude":"lon"})

    #Actual interpolation
#     outarr = inarr.interp(latitude=refarr['latitude'],longitude=refarr['longitude'],method = 'nearest')
    outarr = inarr.interp(lat=refarr['lat'],lon=refarr['lon'],method = 'nearest')

    # Output
    outfname = outfolder + crop + infsuf
    outarr.to_netcdf(outfname)